In [156]:
from pyspark.sql import SparkSession

In [157]:
spark = SparkSession.builder.getOrCreate()

In [158]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType, TimestampType
from pyspark import StorageLevel
import pyspark.sql.functions as F
schema = StructType([StructField('X', DoubleType(), True),StructField('Y', DoubleType(), True),StructField('RowID', IntegerType(), False),
                     StructField('CrimeDateTime', StringType(), True),StructField('CrimeCode', StringType(), True),StructField('Location', StringType(), True),
                     StructField('Description', StringType(), True),StructField('Inside_Outside', StringType(), True),StructField('Weapon', StringType(), True),
                     StructField('Post', StringType(), True),StructField('District', StringType(), True),StructField('Neighborhood', StringType(), True),
                     StructField('Latitude', DoubleType(), True),StructField('Longitude', DoubleType(), True),StructField('GeoLocation', StringType(), True),
                     StructField('Premise', StringType(), True),StructField('VRIName', StringType(), True),StructField('Total_Incidents', IntegerType(), True),
                     StructField('Shape', StringType(), True)])
df = spark.read.option('header', True).schema(schema).csv("Part1_Crime_data.csv")
df.persist(StorageLevel.MEMORY_AND_DISK)
print(df.count())
df.printSchema()
df.show(5)

513158
root
 |-- X: double (nullable = true)
 |-- Y: double (nullable = true)
 |-- RowID: integer (nullable = true)
 |-- CrimeDateTime: string (nullable = true)
 |-- CrimeCode: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Inside_Outside: string (nullable = true)
 |-- Weapon: string (nullable = true)
 |-- Post: string (nullable = true)
 |-- District: string (nullable = true)
 |-- Neighborhood: string (nullable = true)
 |-- Latitude: double (nullable = true)
 |-- Longitude: double (nullable = true)
 |-- GeoLocation: string (nullable = true)
 |-- Premise: string (nullable = true)
 |-- VRIName: string (nullable = true)
 |-- Total_Incidents: integer (nullable = true)
 |-- Shape: string (nullable = true)

+----------------+----------------+-----+--------------------+---------+-------------------+--------------+--------------+-------+----+---------+----------------+--------+---------+------------------+---------------+-------

# Q1

In [159]:
df.select("CrimeCode").distinct().show(1000)

+---------+
|CrimeCode|
+---------+
|       3P|
|       3K|
|      3BJ|
|       1A|
|       3M|
|       5F|
|       4B|
|       3B|
|       7A|
|      3NF|
|      3EF|
|       3N|
|       5D|
|      3LO|
|      3AF|
|       7B|
|      3GO|
|     3AJF|
|      8AO|
|       7C|
|      3AK|
|      3GK|
|       6L|
|      3EO|
|      3JO|
|       3F|
|       1K|
|       8H|
|       4C|
|       5A|
|       6C|
|      3NK|
|       3D|
|       6H|
|      3LK|
|     3AJK|
|      3CO|
|       3L|
|       4E|
|       6D|
|       2A|
|       3C|
|       8I|
|      3NO|
|      3JF|
|       3E|
|      3LF|
|       1O|
|       8J|
|      3CK|
|      8BO|
|       2B|
|      3JK|
|       5B|
|       4A|
|      8GO|
|      8EV|
|      3CF|
|      8EO|
|       6G|
|       6A|
|       9S|
|      3EK|
|      3GF|
|       3H|
|       4D|
|      8FO|
|       6J|
|       6F|
|       6E|
|       3J|
|       5C|
|      8AV|
|       5E|
|     3AJO|
|      3AO|
|       6B|
|      8CV|
|      8BV|
|      8FV|
|   

# Q2

In [160]:
(df.groupBy("CrimeCode").agg(F.count("CrimeCode")).withColumnRenamed("count(CrimeCode)", "CrimeCount")
 .orderBy("CrimeCount", ascending=False).show(1000))

+---------+----------+
|CrimeCode|CrimeCount|
+---------+----------+
|       4E|     91650|
|       6D|     68427|
|       5A|     43928|
|       7A|     40274|
|       6J|     27636|
|       6G|     26858|
|       6E|     24300|
|       6C|     23227|
|       4C|     22438|
|       5D|     14971|
|      3AF|     14739|
|       4B|     14460|
|       4A|     13226|
|       3B|     10737|
|       4D|      7232|
|       5B|      6475|
|       9S|      5443|
|       6F|      5081|
|       5C|      4917|
|       6B|      4106|
|      3CF|      3771|
|       2A|      3168|
|     3AJF|      3018|
|      3AK|      2982|
|       3K|      2833|
|       7C|      2725|
|       1A|      2624|
|      3AO|      2473|
|       5F|      2182|
|      3JF|      1815|
|       3D|      1432|
|       5E|      1419|
|       6L|      1205|
|       8H|      1051|
|       3P|       873|
|      3GF|       789|
|       6A|       781|
|      3BJ|       754|
|      3CK|       591|
|      3CO|       516|
|      3NF|

# Q3

In [161]:
(df.select("Neighborhood").groupBy("Neighborhood").agg(F.count("Neighborhood")).withColumnRenamed("count(Neighborhood)", "CrimeCount")
 .orderBy("CrimeCount", ascending=False).first()["Neighborhood"])

'DOWNTOWN'

# Q4

In [162]:
(df.select(to_timestamp("CrimeDateTime","yyyy/MM/dd HH:mm:ss+00")
           .alias("timestamps")).groupBy(F.month("timestamps")).agg(F.count("timestamps"))
           .orderBy(F.count("timestamps"), ascending=False).show(1))

+-----------------+-----------------+
|month(timestamps)|count(timestamps)|
+-----------------+-----------------+
|                8|            46327|
+-----------------+-----------------+
only showing top 1 row



# Q5

In [163]:
null_values = ["NA","null"]
df.select("Weapon").filter(~F.col("Weapon").isin(null_values)).distinct().show()

+-------+
| Weapon|
+-------+
|  HANDS|
|  KNIFE|
|  OTHER|
|   FIRE|
|FIREARM|
+-------+



# Q6

In [164]:
(df
   .filter(~F.col("Weapon").isin(not_weapon))
   .groupBy("Weapon")
   .agg(F.count("Weapon"))
   .withColumnRenamed("count(Weapon)", "WeaponCount")
   .orderBy("WeaponCount", ascending=False)
   .first()["Weapon"])

'FIREARM'